In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
160,AFG,Asia,Afghanistan,2020-06-18,26874.0,564.0,504.0,13.0,690.345,14.488,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
262,ALB,Europe,Albania,2020-06-18,1722.0,50.0,38.0,1.0,598.374,17.374,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
428,DZA,Africa,Algeria,2020-06-18,11268.0,121.0,799.0,11.0,256.961,2.759,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-17,France,157923.0,0.0,29594.0,0.0,0.0,0.0
0,2020-06-17,China,84501.0,0.0,4656.0,0.0,0.0,0.0
0,2020-06-17,Italy,237931.0,0.0,34455.0,0.0,0.0,0.0
0,2020-06-17,Spain,244581.0,0.0,27146.0,0.0,0.0,0.0
0,2020-06-17,United States,2160629.0,0.0,117555.0,0.0,0.0,0.0
0,2020-06-17,World,8298467.0,0.0,447514.0,0.0,0.0,0.0
0,2020-06-17,United Kingdom,299518.0,0.0,42076.0,0.0,0.0,0.0
0,2020-06-17,Germany,187635.0,0.0,8850.0,0.0,0.0,0.0
0,2020-06-17,Iran,195146.0,0.0,9172.0,0.0,0.0,0.0
0,2020-06-17,Turkey,182769.0,0.0,4859.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-18,France,157923.0,158174.0,29594.0,29575.0,-251.0,19.0
0,2020-06-18,China,84501.0,84458.0,4656.0,4638.0,43.0,18.0
0,2020-06-18,Italy,237931.0,237828.0,34455.0,34448.0,103.0,7.0
0,2020-06-17,Spain,244581.0,244683.0,27146.0,29054.0,-102.0,-1908.0
0,2020-06-18,United States,2160629.0,2163290.0,117555.0,117717.0,-2661.0,-162.0
0,2020-06-18,World,8298467.0,8317573.0,447514.0,450630.0,-19106.0,-3116.0
0,2020-06-18,United Kingdom,299518.0,299251.0,42076.0,42153.0,267.0,-77.0
0,2020-06-18,Germany,187635.0,187764.0,8850.0,8856.0,-129.0,-6.0
0,2020-06-18,Iran,195146.0,195051.0,9172.0,9185.0,95.0,-13.0
0,2020-06-18,Turkey,182769.0,182727.0,4859.0,4861.0,42.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")